# Method of manufactured solutions for mises plasticity

In order to test the implementation of Mises plasticity on a structural level, we use the method of manufactured solutions to obtain an analytical solution for the structural problem. This is achieved by assuming an analytical expression for the displacements which is then used to calculate the strains, stresses and updates of history variables. For the case of linear isotropic hardening, there exists an analytical expression for the stresses which does not require the solution of a nonlinear system of equations. 

The stresses are then used to calculate the forces acting on the structure which are used to determine boundary conditions for an FEM problem. The FEM problem is then solved and the results are compared to the analytical solution.

The analytical expression for the displacement should be chosen such that it cannot be easily expressed by the FEM solution because otherwise the error might become zero and a convergence analysis becomes meaningless. 

Mises plasticity with linear isotropic hardening is given by the following equations:

$$
\begin{aligned}
\sigma^\mathrm{tr} &= \mathbf C : (\varepsilon^{n+1} - \varepsilon^{n}_\mathrm{pl} )\\
s^\mathrm{tr} &= \sigma^\mathrm{tr} - \frac{1}{3}\mathrm{tr}(\sigma^\mathrm{tr})I_3 \\
\sigma^\mathrm{eq} &= \sqrt{\frac{1}{2}s^\mathrm{tr}:s^\mathrm{tr}}\\
f(\sigma) &= \sigma^\mathrm{eq} - \sigma_\mathrm{eq} - h \alpha^n \\
\end{aligned}
$$

Algorithm

1. Determine the trial stress.
2. Determine the equivalent plastic strain increment. $$\Delta\alpha = \frac{\max\{\sigma_\mathrm{eq}-\sigma_0-h\alpha^n,0\}}{3\mu+h}$$ The maximum function serves as the test if the loading is elastic or plastic and therefore $\Delta\alpha$ becomes zero if the loading is elastic.
3. Update the equivalent plastic strain. $$\alpha^{n+1} = \alpha^n + \Delta\alpha$$
4. Update the factor $\gamma$ with the following equation. $$\gamma = 1-\frac{3\mu\Delta\alpha}{\sigma_\mathrm{eq}}$$


In [1]:
from sympy import Function, Matrix, Max, symbols, sqrt, eye, Rational


def strain(u, x, y, z):
    jacobian = u.jacobian([x, y, z])
    return (jacobian + jacobian.T) / 2


def to_mandel_notation(strain):
    factor = sqrt(2)
    return Matrix(
        [
            strain[0, 0],
            strain[1, 1],
            strain[2, 2],
            factor * strain[0, 1],
            factor * strain[0, 2],
            factor * strain[1, 2],
        ]
    )


u, x, y, z = symbols("u x y z")
mu, lam, sig_0, h = symbols(r"\mu \lambda \sigma_0 h")


def elasticity_matrix(mu, lam):
    return Matrix(
        [
            [lam + 2 * mu, lam, lam, 0, 0, 0],
            [lam, lam + 2 * mu, lam, 0, 0, 0],
            [lam, lam, lam + 2 * mu, 0, 0, 0],
            [0, 0, 0, 2 * mu, 0, 0],
            [0, 0, 0, 0, 2 * mu, 0],
            [0, 0, 0, 0, 0, 2 * mu],
        ]
    )


def dev(mandel):
    I_sym = Matrix([1, 1, 1, 0, 0, 0])
    return mandel - (I_sym.dot(mandel) / 3) * I_sym


def J2(mandel):
    return dev(mandel).dot(dev(mandel)) / 2


def equivalent_stress(mandel):
    return sqrt(3 * J2(mandel))


u_x = Function("u_x")(x, y, z)
u_y = Function("u_y")(x, y, z)
u_z = Function("u_z")(x, y, z)
alpha_0 = 0
eps_pl_0 = Matrix([0, 0, 0, 0, 0, 0])
u = Matrix([u_x, u_y, u_z])

J2(elasticity_matrix(mu, lam) * to_mandel_notation(strain(u, x, y, z)))


def delta_alpha(sigma_eq, sig_0, mu, h, alpha_0):
    return Max((sigma_eq - sig_0 - h * alpha_0) / (3 * mu), 0)


def gamma(sigma_eq, mu, delta_alpha):
    return 1 - (3 * mu * delta_alpha) / sigma_eq


def sigma_new(sigma_tr, gamma):
    sigma_dev = dev(sigma_tr)
    return sigma_dev * gamma + sigma_tr.dot(Matrix([1, 1, 1, 0, 0, 0])) / 3 * Matrix(
        [1, 1, 1, 0, 0, 0]
    )


sigma_tr = elasticity_matrix(mu, lam) * to_mandel_notation(strain(u, x, y, z))
sigma_eq = equivalent_stress(sigma_tr)
del_alpha = delta_alpha(sigma_eq, sig_0, mu, h, alpha_0)
gam = gamma(sigma_eq, mu, del_alpha)
sigma_1 = sigma_new(sigma_tr, gam)
alpha_1 = alpha_0 + del_alpha
sigma_1

Matrix([
[2*\lambda*Derivative(u_x(x, y, z), x)/3 + 2*\lambda*Derivative(u_y(x, y, z), y)/3 + 2*\lambda*Derivative(u_z(x, y, z), z)/3 + (\lambda + 2*\mu)*Derivative(u_x(x, y, z), x)/3 + (\lambda + 2*\mu)*Derivative(u_y(x, y, z), y)/3 + (\lambda + 2*\mu)*Derivative(u_z(x, y, z), z)/3 + (-3*\mu*Max(0, (-\sigma_0 + sqrt(12*\mu**2*(Derivative(u_x(x, y, z), y)/2 + Derivative(u_y(x, y, z), x)/2)**2 + 12*\mu**2*(Derivative(u_x(x, y, z), z)/2 + Derivative(u_z(x, y, z), x)/2)**2 + 12*\mu**2*(Derivative(u_y(x, y, z), z)/2 + Derivative(u_z(x, y, z), y)/2)**2 + 3*(-2*\lambda*Derivative(u_x(x, y, z), x)/3 + \lambda*Derivative(u_y(x, y, z), y)/3 + \lambda*Derivative(u_z(x, y, z), z)/3 + 2*(\lambda + 2*\mu)*Derivative(u_x(x, y, z), x)/3 - (\lambda + 2*\mu)*Derivative(u_y(x, y, z), y)/3 - (\lambda + 2*\mu)*Derivative(u_z(x, y, z), z)/3)**2/2 + 3*(\lambda*Derivative(u_x(x, y, z), x)/3 - 2*\lambda*Derivative(u_y(x, y, z), y)/3 + \lambda*Derivative(u_z(x, y, z), z)/3 - (\lambda + 2*\mu)*Derivative(u_x(x,

In [2]:
from sympy import simplify


simplify(sigma_1)

Matrix([
[ ((3*sqrt(6)*\mu*Max(0, (-\sigma_0 + sqrt(\mu**2*(4*Derivative(u_x(x, y, z), x)**2 - 4*Derivative(u_x(x, y, z), x)*Derivative(u_y(x, y, z), y) - 4*Derivative(u_x(x, y, z), x)*Derivative(u_z(x, y, z), z) + 3*Derivative(u_x(x, y, z), y)**2 + 6*Derivative(u_x(x, y, z), y)*Derivative(u_y(x, y, z), x) + 3*Derivative(u_x(x, y, z), z)**2 + 6*Derivative(u_x(x, y, z), z)*Derivative(u_z(x, y, z), x) + 3*Derivative(u_y(x, y, z), x)**2 + 4*Derivative(u_y(x, y, z), y)**2 - 4*Derivative(u_y(x, y, z), y)*Derivative(u_z(x, y, z), z) + 3*Derivative(u_y(x, y, z), z)**2 + 6*Derivative(u_y(x, y, z), z)*Derivative(u_z(x, y, z), y) + 3*Derivative(u_z(x, y, z), x)**2 + 3*Derivative(u_z(x, y, z), y)**2 + 4*Derivative(u_z(x, y, z), z)**2)))/(3*\mu)) - sqrt(18*\mu**2*(Derivative(u_x(x, y, z), y) + Derivative(u_y(x, y, z), x))**2 + 18*\mu**2*(Derivative(u_x(x, y, z), z) + Derivative(u_z(x, y, z), x))**2 + 18*\mu**2*(Derivative(u_y(x, y, z), z) + Derivative(u_z(x, y, z), y))**2 + (\lambda*Derivative(u_x